# <b>Meta Bank</b> Predicción de clientes que se irán

# <b>Paso 1</b> - Preparar los Datos

### <b>1.1</b> - Importar librerias y métodos necesarios

In [6]:
import pandas as pd
# from sklearn.linear_model import LogisticRegression # no me dio los mejores resultados
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

### <b>1.2</b> - Importar datos

In [7]:
data = pd.read_csv('Churn.csv')

### <b>1.3</b> - Exploración inicial de los datos

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [9]:
data.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1


> <b>Observaciones</b><br>
* Las primeras 2 filas no aportan información útil
* Surname es una columna categórica con alta cardinalidad que puede descartarse
* Geography y Gender son columnas que deberan usar OHE
* CreditSCore, Balance y EstimatedSalary son columnas que podrían escalarse
* Tenure tiene alrededor de un 10% de valores nulos

### <b>1.4</b> - Eliminar caracteristicas sin información útil

In [10]:
data = data.drop(columns=['RowNumber', 'CustomerId', 'Surname'])

### <b>1.5</b> - Codificación OHE

In [11]:
characteristics = ['Geography', 'Gender']
data = pd.get_dummies(data, columns=characteristics, drop_first=True)
# reorganizar columnas
data = data[['CreditScore', 'Geography_Germany', 'Geography_Spain', 'Gender_Male', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited']]
# transformar de tipo bool a tipo int
booleans = ['Geography_Germany', 'Geography_Spain', 'Gender_Male']
for col in booleans:
    data[col] = data[col].astype(int)

# <b>Paso 2</b> - Estudio del equilibrio de clases

### <b>2.1</b> - Visualizar equilibrio

In [12]:
# Balance del dataset completo
Exit_bal = data.groupby('Exited')['Age'].count()
print(Exit_bal[1]/Exit_bal[0])

0.25580811252040686


> Las clases estan desbalanceadas 4 A 1, es decir, por cada 5 observaciones 4 son de clientes que no cancelaron y 1 de un cliente que canceló, esto quiere decier que tenemos un desequilibrio en las clases, lo que puede afectar el entrenamiento del modelo

In [13]:
# Balance de los datos con valor nulo en Tenure
tenure_null = data[data['Tenure'].isna()]
tenure_null = tenure_null.groupby('Exited')['Age'].count()
print(tenure_null[1]/tenure_null[0])

0.25206611570247933


> Las proporciones de los datos que tienen un valor ausente en tenure son igualmente proporcionales que en el dataset entero, por que que por esta razon optare en este caso por eliminar las columnas con NaN en Tenure

In [14]:
# Balance después de haber removido los valores nulos de Tenure
data = data.dropna()
Exit_bal = data.groupby('Exited')['Age'].count()
print(Exit_bal[1]/Exit_bal[0])

0.2561835014508774


# <b>Paso 3</b> - Entrenamiento del modelo

    En este caso el modelo se entrenará con los datos preparados pero no en su forma más óptima

### <b>3.1</b> - Separación de los datos

In [15]:
# Separacion de ambos conjuntos
temp, valid = train_test_split(data, test_size=20/100, random_state=1)
train, test = train_test_split(data, test_size=25/100, random_state=1)

# Separacion de caracteristicas y objetivos
train_feat = train.drop(columns='Exited')
train_targ = train['Exited']
valid_feat = valid.drop(columns='Exited')
valid_targ = valid['Exited']
test_feat = test.drop(columns='Exited')
test_targ = test['Exited']

### <b>3.2</b> - Entrenamiento del modelo

In [16]:
# crear instancia del modelo
model = RandomForestClassifier(random_state=1)
# entrenamiento del modelo
model.fit(train_feat, train_targ)

RandomForestClassifier(random_state=1)

### <b>3.3</b> - Validación del modelo

In [17]:
def model_validation(model, train_feat, train_targ, valid_feat, valid_targ, decimals=5):
    """Funcion que calcula varias metricas para un modelo, para dos conjuntos (train y valid)"""
    
    train_pred = model.predict(train_feat)
    valid_pred = model.predict(valid_feat)
    pot = model.predict_proba(train_feat)[:, 1] # Probabilities One Train
    pov = model.predict_proba(valid_feat)[:, 1] # Probabilities One Valid
    
    train_recall = round(recall_score(train_targ, train_pred), decimals)
    train_presi = round(precision_score(train_targ, train_pred), decimals)
    train_f1 = round(f1_score(train_targ, train_pred), decimals)
    train_ROC = round(roc_auc_score(train_targ, pot), decimals)
    valid_recall = round(recall_score(valid_targ, valid_pred), decimals)
    valid_presi = round(precision_score(valid_targ, valid_pred), decimals)
    valid_f1 = round(f1_score(valid_targ, valid_pred), decimals)
    valid_ROC = round(roc_auc_score(valid_targ, pov), decimals)
        
    data = {'train':[train_recall, train_presi, train_f1, train_ROC],
            'valid':[valid_recall, valid_presi, valid_f1, valid_ROC]}
    df = pd.DataFrame(data, index=['recall', 'presision', 'f1', 'roc'])
    
    return df

In [18]:
model_validation(model, train_feat, train_targ, valid_feat, valid_targ)

,train,valid
recall,1.0,0.45225
presision,1.0,0.74194
f1,1.0,0.56195
roc,1.0,0.85459


#### Basándome en estas métricas puedo decir que el modelo:
* Tiene un recal bajo lo que indíca que no esta reconociendo gran parte de las observaciones positivas
* Tiene una presición media lo que significa que no está exagerando el reconocimiento de la clase positiva
* Los puntajes no son similares tanto para train como para valid, por ende, está sobreajustado.
* En general no es bueno y aparentemente la aleatoreidad sería mejor
* Sin embargo el puntaje ROC esta por encima de 0.5 lo que indica que sí es mejor que la simple aleatoreidad

# <b>Paso 4</b> - Mejora del modelo

### <b>4.1</b> - Funcion para sobremuestrear

In [19]:
def upsample(features, target, repetitions):
    """Funcion que agrega repeticiones de valores en la clase positiva"""

    feat_0 = features[target == 0]
    feat_1 = features[target == 1]
    targ_0 = target[target == 0]
    targ_1 = target[target == 1]
    
    feat_up = pd.concat([feat_0] + [feat_1] * repetitions)
    targ_up = pd.concat([targ_0] + [targ_1] * repetitions)
    
    feat_up, targ_up = shuffle(feat_up, targ_up, random_state=1)
    
    return feat_up, targ_up

### <b>4.2</b> - Sobremuestreo de clase positiva

> En un principio el sobremuestreo dio buenos resultados pero los modelos no pudieron rebasar más de 0.59 en el puntaje f1<br>
> y por esta razón el codigo a continuación está comentado
> mas adelante hay un fragmento de código comentado que usé para encontrar el mejor sobremuestreo y le mejor submuestreo, tal código arrojó que, no sobremuestrear, era mejor

In [20]:
# train_feat_up, train_targ_up = upsample(train_feat, train_targ, 2)
# print(len(train_targ_up[train_targ_up == 0]), len(train_targ_up[train_targ_up == 1]))
# model = RandomForestClassifier(random_state=1)
# model.fit(train_feat_up, train_targ_up)
# model_validation(model, train_feat_up, train_targ_up, valid_feat, valid_targ)

### <b>4.3</b> - Funcion para submuestrear

In [21]:
def downsample(features, target, fraction):
    """Funcion que agrega repeticiones de valores en la clase positiva"""

    feat_0 = features[target == 0]
    feat_1 = features[target == 1]
    targ_0 = target[target == 0]
    targ_1 = target[target == 1]
    
    feat_dn = pd.concat([feat_0.sample(frac=fraction, random_state=1)] + [feat_1])
    targ_dn = pd.concat([targ_0.sample(frac=fraction, random_state=1)] + [targ_1])
    
    feat_dn, targ_dn = shuffle(feat_dn, targ_dn, random_state=1)
    
    return feat_dn, targ_dn

### <b>4.4</b> - Submuestreo de clase negativa

In [22]:
train_feat_bal, train_targ_bal = downsample(train_feat, train_targ, fraction=0.59)
print(len(train_targ_bal[train_targ_bal == 0]), len(train_targ_bal[train_targ_bal == 1]))

3194 1405


In [23]:
model = RandomForestClassifier(random_state=1)
model.fit(train_feat_bal, train_targ_bal)
model_validation(model, train_feat_bal, train_targ_bal, valid_feat, valid_targ)

,train,valid
recall,1.0,0.54494
presision,1.0,0.62783
f1,1.0,0.58346
roc,1.0,0.85180


> Las métricas indican que el modelo mejoró

### <b>4.5</b> - Escalado de caracteristicas

In [24]:
bigNums = ['CreditScore', 'Balance', 'EstimatedSalary']

scaler = StandardScaler()
scaler.fit(train_feat_bal[bigNums])

train_feat_bal[bigNums] = scaler.transform(train_feat_bal[bigNums])
valid_feat[bigNums] = scaler.transform(valid_feat[bigNums])

In [25]:
model = RandomForestClassifier(random_state=1)
model.fit(train_feat_bal, train_targ_bal)
model_validation(model, train_feat_bal, train_targ_bal, valid_feat, valid_targ)

,train,valid
recall,1.0,0.54494
presision,1.0,0.62783
f1,1.0,0.58346
roc,1.0,0.85158


> En este caso parece que el escalado de caracteristicas no afectó casi nada pero despues de varias pruebas encontré que sí ayuda en la prueba final.

### <b>4.6</b> - Encontrar las mejores modificaciones para el dataset

In [26]:
# este fragmento de código puede tomar un tiempo considerable
"""
best_upscale = 0 # 1
best_downscale = 0 # 0.59
best_f1 = 0

train_best, valid_best = train_test_split(data, random_state=1, test_size=0.25)
train_best_feat = train_best.drop(columns='Exited')
train_best_targ = train_best['Exited']
valid_best_feat = valid_best.drop(columns='Exited')
valid_best_targ = valid_best['Exited']
train_best_feat[bigNums] = scaler.transform(train_best_feat[bigNums])
valid_best_feat[bigNums] = scaler.transform(valid_best_feat[bigNums])

for ups in range (1, 3):
    for down in range(1, 100, 1):
        tbf = train_best_feat.copy()
        tbt = train_best_targ.copy()
        vbf = valid_best_feat.copy()
        vbt = valid_best_targ.copy()
        # upsample
        tbf, tbt = upsample(tbf, tbt, ups)
        # downsample
        tbf, tbt = downsample(tbf, tbt, down/100)
        # train
        model = RandomForestClassifier(random_state=1)
        model.fit(tbf, tbt)
        result = model_validation(model, tbf, tbt, vbf, vbt)
        
        print(ups, down, result['valid']['f1'])
        if result['valid']['f1'] > best_f1:
            best_f1 = result['valid']['f1']
            best_upscale = ups
            best_downscale = down/100
            print(f'upsample: {best_upscale}, downsample: {best_downscale}, f1: {best_f1}')
"""
# los mesjores valores que encontro para modificar las caracteristicas fueron:
# ups 1, down, 0.59
print()# para evitar que aparezca el codigo abajo como mensaje

# <b>Paso 5</b> - Prueba Final

In [27]:
test_feat[bigNums] = scaler.transform(test_feat[bigNums])
model_results = model_validation(model, train_feat_bal, train_targ_bal, test_feat, test_targ)
model_results

,train,valid
recall,1.0,0.57461
presision,1.0,0.63861
f1,1.0,0.60492
roc,1.0,0.85750
